In [21]:
# load gt
import json

path = '../../dataset/qrecc_test.json'

with open(path) as f: 
    data = json.load(f)

In [55]:
# load runs to check model answers
import os

runs = []
model_labels = []

path = '../../runs/'
for filename in os.listdir(path):
    with open(path+filename, encoding="utf-8") as f:
        print(filename)
        run = json.load(f)
        runs.append(run)
        model_labels.append(filename.split('.')[0])
        
print('\n %d runs loaded' % len(runs))

simple_baseline_rewritten.json
gpt3_run.json
scai-qrecc21-naacl-baseline.json

 3 runs loaded


In [39]:
# load answer encoder
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('paraphrase-MiniLM-L12-v2')

In [62]:
limit = 5
for i, sample in enumerate(data[:limit]):
    # group all answers
    gt_answers = [sample['Answer']]
    model_answers = []
    for j in range(len(runs)):
        assert sample['Conversation_no'] == runs[j][i]['Conversation_no'] and sample['Turn_no'] == runs[j][i]['Turn_no']
        model_answers.append(runs[j][i]['Model_answer'])
    
    #Compute embedding for both lists
    gt_embeddings = model.encode(gt_answers, convert_to_tensor=True)
    model_embeddings = model.encode(model_answers, convert_to_tensor=True)

    #Compute cosine-similarits
    cosine_scores = util.pytorch_cos_sim(gt_embeddings, model_embeddings)
    
    #Find the pairs with the highest cosine similarity scores
    pairs = []
    for i in range(len(cosine_scores)):
        for j in range(len(cosine_scores[i])):
            pairs.append({'index': [i, j], 'score': cosine_scores[i][j]})
    
    #Sort scores in decreasing order
    pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)
    
    print(gt_answers)
    for pair in pairs[0:10]:
        i, j = pair['index']
        print("{} \t\t {} \t\t Score: {:.4f}".format(model_labels[j], model_answers[j], pair['score']))
    print('\n')

['physician assistants are medical providers who are licensed to diagnose and treat illness and disease and to prescribe medication for patients']
gpt3_run 		  A physician's assistant is a person who assists a physician in the performance of his or her duties. 		 Score: 0.8473
scai-qrecc21-naacl-baseline 		 Physician assistant 		 Score: 0.7900
simple_baseline_rewritten 		 The Physician Assistant Board or their representative may require proof or demonstration of competence from any physician assistant for any medical services performed. 		 Score: 0.6940


["Complete your bachelor's degree (a science or healthcare related major is usually best); Gain experience either working or volunteering in a healthcare setting; Apply to ARC-PA accredited physician assistant programs; Complete a 2-3 year, master's level PA program;"]
scai-qrecc21-naacl-baseline 		 a physician assistant will need a master’s degree from an accredited institution (two years of post-graduate education after completing a